In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

# use seaborn plotting defaults
import seaborn as sns; sns.set()

In [3]:
import requests

In [5]:
url = "https://api.github.com/repos/pandas-dev/pandas/issues"

r = requests.get(url)
r.raise_for_status
r

<Response [200]>

In [9]:
data = r.json()
data[0]['number'] - data[-1]['number']

33

In [12]:
issues = pd.DataFrame(data, columns=["number", "title",
                                      "labels", "state"])
issues.tail()

,number,title,labels,state
25,56521,DEPR: groupby.grouper,"[{'id': 233160, 'node_id': 'MDU6TGFiZWwyMzMxNj...",open
26,56520,CoW: Set a bunch of Series copy statements to ...,[],open
27,56519,TYP: make dtype required in _from_sequence_of_...,[],open
28,56518,DEPR: SparseDtype,"[{'id': 49182326, 'node_id': 'MDU6TGFiZWw0OTE4...",open
29,56516,ENH: remove unused entries from _lite_rule_alias,"[{'id': 53181044, 'node_id': 'MDU6TGFiZWw1MzE4...",open


In [13]:
issues['state'].value_counts()

state
open    30
Name: count, dtype: int64